In [1]:
import numpy as np
from scipy.linalg import eigh

In [3]:
def single_view_graph_learning(X, c, beta=1.0, max_iter=100):
    '''
    X: 单视图数据，维度为 (样本数n x 特征维度d)
    c: 聚类簇数
    beta: 正则化参数
    '''
    n = X.shape[0]

    # 初始化S矩阵
    S = np.zeros((n, n))

    # 初始化Q矩阵，可以用数据的前c个特征向量初始化
    Q = np.linalg.qr(np.random.rand(n, c))[0]

    for iter in range(max_iter):
        # Step 1: 更新S，固定Q
        for j in range(n):
            q_diff = np.linalg.norm(Q - Q[j, :], axis=1)**2
            s_j = (-q_diff / (2*beta))
            s_j = np.exp(s_j - np.max(s_j))  # 防止数值溢出
            s_j /= np.sum(s_j)
            S[:, j] = s_j

        # Step 2: 更新Q，固定S
        S_sym = (S + S.T) / 2
        D = np.diag(np.sum(S_sym, axis=1))
        L = D - S_sym

        eigvals, eigvecs = eigh(L, subset_by_index=[0, c-1])
        Q = eigvecs

        # 检查连通分量个数是否达到聚类数c
        eigen_gap = np.sum(eigvals[:c])
        if eigen_gap < 1e-10:
            break

    return S